<h1>Computing Coursework 2024</h1>
<h5>Michael Pan, Joshua Chong</h5>

<h3>Importing Libraries</h3>

In [4]:
# make sure required libraries are installed
!pip install numpy matplotlib
%matplotlib inline

# In this section I am importing all the libraries I will need
import numpy as np
import matplotlib.pyplot as plt


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\Michael\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


### Black-Scholes Equation
The Black-Scholes equation for options value is defined by
$$ \frac{\partial V}{\partial t} + \frac{1}{2}\sigma^2S^2\frac{\partial^2 V}{\partial S^2} + rS\frac{\partial V}{\partial S} - rV = 0 $$
where $V$ is price of the call option, $S$ is price of the underlying stock, $t$ is the time in years, $r$ is the risk-free interest rate, and $\sigma$ is the volatility of the underlying stock price. 

To solve this PDE numerically, we transformed into the form of the heat diffusion equation 
$$ \frac{\partial u}{\partial \tau} = \frac{\sigma^2}{2}\frac{\partial^2 u}{\partial x^2} $$
using the transformations
$$ \tau = T - t $$
$$ x = \ln{\frac{S}{K}} $$
$$ U(x,\tau) = Ve^{(\frac{r}{\sigma^2}-\frac{1}{2})x + (\frac{r^2}{2\sigma^2} + \frac{r}{2} + \frac{\sigma^2}{8})\tau} $$

### Setting Constants

In [ ]:
# Expiration time in years
T = 1
# Strike price 
K = 100
# Risk-free interest rate
r = 0.05
# Volatility of stock price
sigma = 0.2



### Discretised Grid

In [ ]:
# In this section I am setting the domain of solution and the discretised grid
# The domain of x
L = 200


In [ ]:
# In this section I am defining arrays I would need (if neeeded)





In [ ]:
# In this section I am setting the boundary conditions/initial values






In [ ]:
# In this section I am implementing the numerical method






In [ ]:
# In this section I am showing the results





In [ ]:
# In this section I am celebrating
print('CW done: I deserve a good mark')